# Calculate undetected peptides and apply SpC-NSAF

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Notebook summary:
1. Import fasta peptides and detected peptides datasets.
2. Dataset cleaning.
    - Sort detected peptides by PEP score.
    - Remove duplicates in both datasets.
    - Only keep proteins in fasta peptides that are also present in detected peptides - this is to be the expected peptides.
3. Get undetected peptides.
    - Remove all rows in expected peptides that are also present in detected peptides.
    - Calculate SpC-NSAF for undetected peptides
4. Export a new data frame with undetected peptides as a TSV file.

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import csv

# set display options
#pd.set_option("display.max_rows", None, "display.max_columns", None)

# 1. Import fasta peptides and detected peptides

In [ ]:
# import datasets
fasta_peptides = pd.read_table('/content/drive/MyDrive/Colab Notebooks/peptide ml detection/data/fasta_peptides.tsv')
detected_peptides = pd.read_table('/content/drive/MyDrive/Colab Notebooks/peptide ml detection/data/detected_peptides_NSAF.tsv')

In [ ]:
fasta_peptides.shape

(209525, 6)

In [ ]:
detected_peptides.shape

(1917, 6)

# 2. Dataset cleaning

### get expected peptides before removing duplicates

In [ ]:
# initial check on how many total proteins in detected_peptides are not in fasta_peptides (should be 0)
len(detected_peptides[~detected_peptides["Protein"].isin(fasta_peptides["Protein"])])

573

In [ ]:
# initial check on how many total peptides (i.e. PSMs) in detected_peptides are not in fasta_peptides (should be 0)
len(detected_peptides[~detected_peptides["Peptide"].isin(fasta_peptides["Peptide"])])
# NB: These sequences are actually there, they appear to be start of the sequences
# i.e. all these sequences start with Methionine in the fasta peptides.

1819

In [ ]:
# initial check on how many total proteins and peptides in fasta_peptides are not in detected_peptides
len(fasta_peptides[~fasta_peptides["Protein"].isin(detected_peptides["Protein"])])

169723

In [ ]:
# check how many protein+peptide in detected_peptides are present in fasta_peptides (should be 37,847)
len(detected_peptides.set_index(['Protein', 'Peptide']).index.isin(fasta_peptides.set_index(['Protein', 'Peptide']).index))

1917

In [ ]:
len(fasta_peptides['Protein'])

209525

## keep proteins in fasta peptides that are present in detected peptides -expected peptides

In [ ]:
detected_peptides.describe(include='all')

,Protein,Peptide,PEP,Protein_length,PSM_per_protein,Quantification
count,1917,1917,1917.000000,1344.000000,1917.000000,1344.000000
unique,1104,1917,NaN,NaN,NaN,NaN
top,Q09666,YYDSRPGGYGYGYGR,NaN,NaN,NaN,NaN
freq,40,1,NaN,NaN,NaN,NaN
mean,NaN,NaN,0.011226,877.968006,25.293166,0.047458
std,NaN,NaN,0.025251,1078.848877,44.137130,0.076706
min,NaN,NaN,0.004787,80.000000,1.000000,0.000393
25%,NaN,NaN,0.004788,314.000000,5.000000,0.012469
50%,NaN,NaN,0.004826,546.500000,11.000000,0.029112
75%,NaN,NaN,0.005565,940.750000,25.000000,0.057656


In [ ]:
# initial check on how many total proteins and peptides in detected_peptides are in fasta_peptides
# (should be 37,847 if after removing duplicate peptides)
len(detected_peptides[detected_peptides["Protein"].isin(fasta_peptides["Protein"])])

1344

In [ ]:
# initial check on how many total proteins in fasta_peptides are in detected_peptides
len(fasta_peptides[fasta_peptides["Protein"].isin(detected_peptides["Protein"])])

39802

In [ ]:
# remove proteins in fasta_peptides not present in detected_peptides - let this be expected_peptides
expected_peptides = fasta_peptides[fasta_peptides["Protein"].isin(detected_peptides["Protein"])]
expected_peptides.describe(include="all")

,Protein,Peptide,Cleavage,Prediction,Probability,Length
count,39802,39802,39802.000000,39802.0,39802.000000,39802.000000
unique,659,6182,NaN,NaN,NaN,NaN
top,Q09666,K,NaN,NaN,NaN,NaN
freq,909,3084,NaN,NaN,NaN,NaN
mean,NaN,NaN,0.170544,1.0,0.736235,892.267775
std,NaN,NaN,0.376115,0.0,0.118713,957.390632
min,NaN,NaN,0.000000,1.0,0.500000,80.000000
25%,NaN,NaN,0.000000,1.0,0.642000,379.000000
50%,NaN,NaN,0.000000,1.0,0.779000,612.000000
75%,NaN,NaN,0.000000,1.0,0.817000,1042.000000


# 3. Get undetected peptides

In [ ]:
# initial check on undetected peptides
undetected_peptides = pd.merge(expected_peptides, detected_peptides,
                               on=["Peptide"],
                               how='left', indicator=True)
undetected_peptides

,Protein_x,Peptide,Cleavage,Prediction,Probability,Length,Protein_y,PEP,Protein_length,PSM_per_protein,Quantification,_merge
0,P18031,EFEQIDK,1,1,0.500,435,NaN,NaN,NaN,NaN,NaN,left_only
1,Q96P70,AIITDSVK,1,1,0.500,1041,NaN,NaN,NaN,NaN,NaN,left_only
2,Q96P70,AIITDSVK,1,1,0.500,1041,NaN,NaN,NaN,NaN,NaN,left_only
3,P18031,EFEQIDK,1,1,0.500,435,NaN,NaN,NaN,NaN,NaN,left_only
4,P33176,SAEIDSDDTGGSAAQK,1,1,0.500,963,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...
39797,Q9NZ45,IVHAFDMEDLGDK,0,1,0.981,108,NaN,NaN,NaN,NaN,NaN,left_only
39798,Q92541,K,0,1,0.981,710,NaN,NaN,NaN,NaN,NaN,left_only
39799,Q92541,QEEEQEK,0,1,0.981,710,NaN,NaN,NaN,NaN,NaN,left_only
39800,P00558,FHVEEEGK,0,1,0.981,417,NaN,NaN,NaN,NaN,NaN,left_only


In [ ]:
# find peptides/PSMs present only in undetected_peptides
len(undetected_peptides[undetected_peptides['_merge'].str.contains("left_only", na=False)])

39223

In [ ]:
# find number of undetected using isin
len(expected_peptides[~expected_peptides["Peptide"].isin(detected_peptides["Peptide"])])

39223

In [ ]:
undetected_peptides = expected_peptides[~expected_peptides["Peptide"].isin(detected_peptides["Peptide"])]
undetected_peptides.shape

(39223, 6)

In [ ]:
undetected_peptides.describe(include="all")

,Protein,Peptide,Cleavage,Prediction,Probability,Length
count,39223,39223,39223.000000,39223.0,39223.000000,39223.000000
unique,659,6084,NaN,NaN,NaN,NaN
top,Q09666,K,NaN,NaN,NaN,NaN
freq,858,3084,NaN,NaN,NaN,NaN
mean,NaN,NaN,0.172093,1.0,0.736269,889.170232
std,NaN,NaN,0.377466,0.0,0.118645,945.419437
min,NaN,NaN,0.000000,1.0,0.500000,80.000000
25%,NaN,NaN,0.000000,1.0,0.642000,379.000000
50%,NaN,NaN,0.000000,1.0,0.779000,612.000000
75%,NaN,NaN,0.000000,1.0,0.817000,1042.000000


In [ ]:
# remove any peptides containing 'U' (selenocysteine) amino acids
# this is so that we can map peptide amino acids correctly when calculating AAIndex1 properties
len(undetected_peptides[undetected_peptides['Peptide'].str.contains("U", na=False)])

0

In [ ]:
undetected_peptides[undetected_peptides['Peptide'].str.contains("U", na=False)]

,Protein,Peptide,Cleavage,Prediction,Probability,Length


In [ ]:
undetected_peptides.shape

(39223, 6)

In [ ]:
len(undetected_peptides[~undetected_peptides['Peptide'].str.contains("U", na=False)])

39223

In [ ]:
undetected_peptides = undetected_peptides[~undetected_peptides['Peptide'].str.contains("U", na=False)]
undetected_peptides.shape

(39223, 6)

# Calculate SpC-NSAF for undetected peptides

In [ ]:
# check for any intersection between detected and undetected (should be 0)
print(len(set(detected_peptides["Peptide"]).intersection(set(undetected_peptides["Peptide"]))))

0


In [ ]:
# create dictionary from detected_peptides, with protein keys and associated NSAF values
detected_peptides_nsaf_dict = dict(zip(detected_peptides.Protein, detected_peptides.Quantification))

In [ ]:
# map protein NSAF quantitation values to each protein in undetected_peptides
undetected_peptides['Quantification'] = undetected_peptides['Protein'].map(detected_peptides_nsaf_dict)
undetected_peptides

,Protein,Peptide,Cleavage,Prediction,Probability,Length,Quantification
0,P18031,EFEQIDK,1,1,0.500,435,0.022989
18,Q96P70,AIITDSVK,1,1,0.500,1041,0.025937
19,Q96P70,AIITDSVK,1,1,0.500,1041,0.025937
20,P18031,EFEQIDK,1,1,0.500,435,0.022989
24,P33176,SAEIDSDDTGGSAAQK,1,1,0.500,963,0.045691
...,...,...,...,...,...,...,...
209482,Q9NZ45,IVHAFDMEDLGDK,0,1,0.981,108,0.185185
209493,Q92541,K,0,1,0.981,710,0.008451
209494,Q92541,QEEEQEK,0,1,0.981,710,0.008451
209502,P00558,FHVEEEGK,0,1,0.981,417,0.019185


In [ ]:
# check for any NaN values
undetected_peptides.isnull().sum()

Protein           0
Peptide           0
Cleavage          0
Prediction        0
Probability       0
Length            0
Quantification    0
dtype: int64

In [ ]:
print(undetected_peptides.shape)
print(detected_peptides.shape)

(39223, 7)
(1917, 6)


In [ ]:
undetected_peptides.describe(include='all')

,Protein,Peptide,Cleavage,Prediction,Probability,Length,Quantification
count,39223,39223,39223.000000,39223.0,39223.000000,39223.000000,39223.000000
unique,659,6084,NaN,NaN,NaN,NaN,NaN
top,Q09666,K,NaN,NaN,NaN,NaN,NaN
freq,858,3084,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,0.172093,1.0,0.736269,889.170232,0.033220
std,NaN,NaN,0.377466,0.0,0.118645,945.419437,0.055352
min,NaN,NaN,0.000000,1.0,0.500000,80.000000,0.000393
25%,NaN,NaN,0.000000,1.0,0.642000,379.000000,0.007463
50%,NaN,NaN,0.000000,1.0,0.779000,612.000000,0.018657
75%,NaN,NaN,0.000000,1.0,0.817000,1042.000000,0.045671


In [ ]:
# remove any peptides in undetected_peptides that map to more than one different protein
undetected_peptides_clean = undetected_peptides.groupby('Peptide').filter(lambda x: x['Protein'].nunique() == 1)
undetected_peptides_clean.shape

(34515, 7)

In [ ]:
undetected_peptides_clean.describe(include='all')

,Protein,Peptide,Cleavage,Prediction,Probability,Length,Quantification
count,34515,34515,34515.000000,34515.0,34515.000000,34515.000000,34515.000000
unique,659,6052,NaN,NaN,NaN,NaN,NaN
top,Q09666,APFDLFENK,NaN,NaN,NaN,NaN,NaN
freq,731,153,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,0.188527,1.0,0.736493,890.411415,0.031413
std,NaN,NaN,0.391138,0.0,0.118288,934.541117,0.044673
min,NaN,NaN,0.000000,1.0,0.500000,80.000000,0.000393
25%,NaN,NaN,0.000000,1.0,0.644000,382.000000,0.007394
50%,NaN,NaN,0.000000,1.0,0.779000,624.000000,0.018033
75%,NaN,NaN,0.000000,1.0,0.816000,1046.000000,0.044855


In [ ]:
# drop duplicate peptides (i.e. if a peptide sequence that matches to same protein occurs twice or more), keep first occurrence
undetected_peptides_clean = undetected_peptides_clean.drop_duplicates(subset = ['Peptide', 'Protein'],
                                                                      keep = "first").reset_index(drop = True)
undetected_peptides_clean.describe(include="all")

,Protein,Peptide,Cleavage,Prediction,Probability,Length,Quantification
count,6052,6052,6052.000000,6052.0,6052.000000,6052.000000,6052.000000
unique,659,6052,NaN,NaN,NaN,NaN,NaN
top,Q92616,EFEQIDK,NaN,NaN,NaN,NaN,NaN
freq,73,1,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,0.079808,1.0,0.635305,847.549736,0.030239
std,NaN,NaN,0.271019,0.0,0.116417,799.114566,0.042953
min,NaN,NaN,0.000000,1.0,0.500000,80.000000,0.000393
25%,NaN,NaN,0.000000,1.0,0.523000,379.750000,0.007676
50%,NaN,NaN,0.000000,1.0,0.590000,620.000000,0.017167
75%,NaN,NaN,0.000000,1.0,0.756000,1014.000000,0.040548


In [ ]:
# check how many detected proteins are not in undetected
len(detected_peptides[~detected_peptides["Protein"].isin(undetected_peptides_clean["Protein"])])

573

In [ ]:
detected_peptides[~detected_peptides["Protein"].isin(undetected_peptides_clean["Protein"])]

,Protein,Peptide,PEP,Protein_length,PSM_per_protein,Quantification
2,P62851,DKLNNLVLFDK,0.004787,NaN,10,NaN
13,P31949,NQKDPGVLDR,0.004787,NaN,27,NaN
17,Q8WXX5,QKEMDNFLAQMEAK,0.004787,NaN,13,NaN
21,P09669,NYDVMKDFEEMR,0.004787,NaN,41,NaN
22,Q5VTE6,NWEYICSHDKEK,0.004787,NaN,17,NaN
...,...,...,...,...,...,...
1907,P30414,VRPGSLFDEVR,0.195430,NaN,4,NaN
1909,C9JU14,YLEDRTVR,0.210640,NaN,1,NaN
1910,Q9Y490,DLQEVKAAAR,0.212410,NaN,67,NaN
1911,Q9NTZ6,DEATAAVIDLNDRPIGSR,0.227690,NaN,12,NaN


In [ ]:
# initial check on how many total proteins and PSMs in detected_peptides are in fasta_peptides
len(detected_peptides[detected_peptides["Protein"].isin(fasta_peptides["Protein"])])

1344

In [ ]:
undetected_peptides_clean.shape

(6052, 7)

# 4. Export a new data frame- undetected peptides as a TSV file

In [ ]:
# export undetected peptides to TSV
undetected_peptides_clean.to_csv("/content/drive/MyDrive/Colab Notebooks/peptide ml detection/data/undetected_peptides_NSAF.tsv", sep='\t', index=False)